In [1]:
import tensorflow as tf
import numpy as np

2024-07-05 14:55:35.736354: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 14:55:36.266259: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 14:55:36.269416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-05 14:55:37.650830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dtype = tf.float16 # subject to change
BASE_DIR = "/home/kernal1/QM_Sandbox/Phi_2/phi_local/weights/"
DECODER_LAYERS = 32
DECODER_RANGE = [0, 0]

In [3]:
params = {}
# params['embed_tokens'] = tf.constant(np.load(BASE_DIR + 'embed_tokens.npy'), dtype=dtype)
params['decoder_layers'] = dict()

# for i in range(DECODER_LAYERS):
for i in range(*DECODER_RANGE):
  layer_params = {}
  layer_params['layernorm_weight'] = 'layernorm_weight.npy'
  layer_params['layernorm_bias'] = 'layernorm_bias'
  layer_params['q_proj_weight'] = 'q_proj_weight'
  layer_params['q_proj_bias'] = 'q_proj_bias'
  layer_params['k_proj_weight'] = 'k_proj_weight'
  layer_params['k_proj_bias'] = 'k_proj_bias'
  layer_params['v_proj_weight'] = 'v_proj_weight'
  layer_params['v_proj_bias'] = 'v_proj_bias'
  layer_params['dense_weight'] = 'dense_weight'
  layer_params['dense_bias'] = 'dense_bias'
  layer_params['mlp_fc1_weight'] = 'mlp_fc1_weight'
  layer_params['mlp_fc1_bias'] = 'mlp_fc1_bias'
  layer_params['mlp_fc2_weight'] = 'mlp_fc2_weight'
  layer_params['mlp_fc2_bias'] = 'mlp_fc2_bias'
  for key in layer_params:
    layer_params[key] = tf.constant(np.load(BASE_DIR + str(i) + '_' + key + '.npy'), dtype=dtype)
    if "weight" in key and "layernorm" not in key:
      layer_params[key] = tf.transpose(layer_params[key], perm=[1,0])
    print(key, ": ", layer_params[key].shape) # remove later
  # params['decoder_layers'].append(layer_params)
  params['decoder_layers'][i] = layer_params

params['final_layernorm_weight'] = tf.constant(np.load(BASE_DIR + 'final_layernorm_weight.npy'), dtype=dtype)
params['final_layernorm_bias'] = tf.constant(np.load(BASE_DIR + 'final_layernorm_bias.npy'), dtype=dtype)

params['lm_head_weight'] = tf.transpose(tf.constant(np.load(BASE_DIR + 'lm_head_weight.npy'), dtype=dtype), perm=[1,0])
params['lm_head_bias'] = tf.constant(np.load(BASE_DIR + 'lm_head_bias.npy'), dtype=dtype)

2024-07-05 14:55:40.540722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-05 14:55:40.544137: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
class PhiConfig():
    keys_to_ignore_at_inference = ["past_key_values"]
    def __init__(
        self,
        vocab_size=51200,
        hidden_size=2560,
        intermediate_size=10240,
        num_hidden_layers=DECODER_LAYERS, # modified,
        num_attention_heads=32,
        num_key_value_heads=32,
        resid_pdrop=0.0, # the model bt default has 0.1 on colab, but we dont have randomness
        embd_pdrop=0.0,
        attention_dropout=0.0,
        hidden_act="gelu_new",
        max_position_embeddings=2048,
        initializer_range=0.00,
        layer_norm_eps=1e-5,
        use_cache=True,                  # Modifed
        tie_word_embeddings=False,
        rope_theta=10000.0,
        rope_scaling=None,
        partial_rotary_factor=0.4,
        qk_layernorm=False,
        bos_token_id=50256,
        eos_token_id=50256,
    ):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads

        if num_key_value_heads is None:
            num_key_value_heads = num_attention_heads

        self.num_key_value_heads = num_key_value_heads
        self.resid_pdrop = resid_pdrop
        self.embd_pdrop = embd_pdrop
        self.attention_dropout = attention_dropout
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.layer_norm_eps = layer_norm_eps
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.partial_rotary_factor = partial_rotary_factor
        self.qk_layernorm = qk_layernorm
        self.pad_token_id = None
        self._attn_implementation = 'eager'

In [5]:
# upcasting from fp16 to fp32
# for key in params:
#     if (key != 'decoder_layers'):
#         params[key] = tf.cast(params[key], dtype=tf.float32)

# for index in params['decoder_layers']:
#     for key in params['decoder_layers'][index]:
#         params['decoder_layers'][index][key] = tf.cast(params['decoder_layers'][index][key], dtype=tf.float32)

In [6]:
config = PhiConfig(use_cache=True)
import importlib

import split_defs

In [7]:
def saveTensor(tensors:list, base_name:str, dir='.'):
    for i in range(len(tensors)):
        tensors[i].numpy().tofile(dir + "/" + base_name + "_" + str(i) + ".bin")

def saveTensorSingle(tensor:list, base_name:str, dir='.'):
        tensor.numpy().tofile(dir + "/" + base_name + ".bin")

def loadTensor(path, shape:list, dtype):
    with open(path, 'rb') as f:
        tensor = np.fromfile(f, dtype=dtype)
        tensor = tensor.reshape(shape)
        return tensor

In [8]:
# for saving the layernorms
# for i in range(*DECODER_RANGE):
#     saveTensor((params['decoder_layers'][i]['layernorm_weight'],), 'layernorm_weight_' + str(i), "data")
#     saveTensor((params['decoder_layers'][i]['layernorm_bias'],), 'layernorm_bias_' + str(i), "data")


# assert(params['final_layernorm_weight'].dtype == tf.float32)
# assert(params['final_layernorm_bias'].dtype == tf.float32)
# assert(params['lm_head_weight'].dtype == tf.float32)
# assert(params['lm_head_bias'].dtype == tf.float32)

# saveTensorSingle(params['final_layernorm_weight'], "final_layernorm_weight", "data")
# saveTensorSingle(params['final_layernorm_bias'], "final_layernorm_bias", "data")
# saveTensorSingle(params['lm_head_weight'], "lm_head_weight", "data")
# saveTensorSingle(params['lm_head_bias'], "lm_head_bias", "data")

In [9]:
# testing thing, ignore this block
# one = 1
# # TOT_SEQ_LEN = 11
# importlib.reload(split_defs)
# from split_defs import *
# model_P3_not_first_reshaped = PhiDecodeP3_not_first_reshaped_test()
# # tf.saved_model.save(model_P3_not_first_reshaped, "tf/model_P3_not_first_reshaped_test")
# i1 = tf.random.uniform([1, 32, 2], dtype=tf.float32)
# i2 = tf.random.uniform([1, 32, 80], dtype=tf.float32)
# # saveTensor([i1, i2], "P3_not_first_reshaped", "tf")
# model_P3_not_first_reshaped(i1, i2)['attn_output'].shape

# SMALL_SIZE = 17
# MEDIUM_SIZE = 23
# params = {}
# params['layernorm_weight'] = tf.random.uniform([SMALL_SIZE])
# params['layernorm_bias'] = tf.random.uniform([SMALL_SIZE])
# for char in ['q', 'k', 'v']:
#     params[char + '_proj_weight'] = tf.random.uniform([SMALL_SIZE, SMALL_SIZE])
#     params[char + '_proj_bias'] = tf.random.uniform([SMALL_SIZE])
# params['mlp_fc1_weight'] = tf.random.uniform([SMALL_SIZE, MEDIUM_SIZE])
# params['mlp_fc1_bias'] = tf.random.uniform([MEDIUM_SIZE])
# params['mlp_fc2_weight'] = tf.random.uniform([MEDIUM_SIZE, SMALL_SIZE])
# params['mlp_fc2_bias'] = tf.random.uniform([SMALL_SIZE])
# config_test = config
# config_test.hidden_size = SMALL_SIZE
# config_test.intermediate_size = MEDIUM_SIZE

# model_P1_reshaped_test = PhiDecodeP1_reshaped_test(config_test, params)
# tf.saved_model.save(model_P1_reshaped_test, "tf/model_P1_reshaped_test")


In [10]:
# testing each part
importlib.reload(split_defs)
from split_defs import *

# model_gelu = NewGELU()
# tf.saved_model.save(model_gelu, "tf/model_gelu")
# i1 = tf.random.uniform([SEQ_LEN, INTERMEDIATE_SIZE], dtype=tf.float32)
# saveTensor([i1], "gelu", "tf")
# model_gelu(i1)

# for i in range(DECODER_LAYERS):
# for i in range(*DECODER_RANGE):
    # model_P1_Q_reshaped = PhiDecodeP1_Q_reshaped_new_quant(params['decoder_layers'][i])
    # tf.saved_model.save(model_P1_Q_reshaped, "tf/model_P1_Q_reshaped_layer_" + str(i))
    # i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
    # # saveTensor([i1], "P1_Q_reshaped", "tf")
    # model_P1_Q_reshaped(i1)

    # model_P1_K_reshaped = PhiDecodeP1_K_reshaped_new_quant(params['decoder_layers'][i])
    # tf.saved_model.save(model_P1_K_reshaped, "tf/model_P1_K_reshaped_layer_" + str(i))
    # i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
    # # saveTensor([i1], "P1_K_reshaped", "tf")
    # model_P1_K_reshaped(i1)

    # model_P1_V_reshaped = PhiDecodeP1_V_reshaped_new_quant(params['decoder_layers'][i])
    # tf.saved_model.save(model_P1_V_reshaped, "tf/model_P1_V_reshaped_layer_" + str(i))
    # i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
    # # saveTensor([i1], "P1_V_reshaped", "tf")
    # model_P1_V_reshaped(i1)

    # model_P1_FC1_reshaped = PhiDecodeP1_FC1_reshaped_new_quant(config, params['decoder_layers'][i])
    # tf.saved_model.save(model_P1_FC1_reshaped, "tf/model_P1_FC1_reshaped_layer_" + str(i))
    # i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
    # # saveTensor([i1], "P1_FC1_reshaped", "tf")
    # model_P1_FC1_reshaped(i1)

    # model_P1_2_reshaped = PhiDecodeP1_2_reshaped_new_quant(config, params['decoder_layers'][i])
    # tf.saved_model.save(model_P1_2_reshaped, "tf/model_P1_2_reshaped_layer_" + str(i))
    # i1 = tf.random.uniform([SEQ_LEN, INTERMEDIATE_SIZE], dtype=tf.float32)
    # saveTensor([i1], "P1_2_reshaped", "tf")
    # model_P1_2_reshaped(i1)

# model_P2_1_first_buffered = PhiDecodeP2_1_first_buffered_unquant_fp32(config)
# # tf.saved_model.save(model_P2_1_first_buffered, "tf/model_P2_1_first_buffered")
# i1 = tf.random.uniform([32, MAX_SEQ_LEN, 80], dtype=tf.float32)
# i2 = tf.random.uniform([32, MAX_SEQ_LEN, 80], dtype=tf.float32)
# i3 = tf.random.uniform([MAX_SEQ_LEN, MAX_SEQ_LEN], dtype=tf.float32)
# # saveTensor([i1, i2, i3], "P2_1_first_buffered", "tf")
# model_P2_1_first_buffered(i1, i2, i3)

# model_P2_not_first_reshaped = PhiDecodeP2_not_first_reshaped_unquant_fp32(config)
# # tf.saved_model.save(model_P2_not_first_reshaped, "tf/model_P2_not_first_reshaped")
# i1 = tf.random.uniform([SEQ_LEN, 32, 80], dtype=tf.float32)
# i2 = tf.random.uniform([TOT_SEQ_LEN, 32, 80], dtype=tf.float32)
# i3 = tf.random.uniform([TOT_SEQ_LEN], dtype=tf.float32)
# # saveTensor([i1, i2, i3], "P2_not_first_reshaped", "tf")
# model_P2_not_first_reshaped(i1, i2, i3)

# model_P2_not_first_reshaped = PhiDecodeP2_not_first_reshaped_unquant_fp32(config)
# # tf.saved_model.save(model_P2_not_first_reshaped, "tf/model_P2_not_first_reshaped")
# i1 = tf.random.uniform([SEQ_LEN, 32, 80], dtype=tf.float32)
# i2 = tf.random.uniform([TOT_SEQ_LEN, 32, 80], dtype=tf.float32)
# # saveTensor([i1, i2], "P2_not_first_reshaped", "tf")
# model_P2_not_first_reshaped(i1, i2)

model_P2_reshaped = PhiDecodeP2_reshaped_unquant_fp32(config)
tf.saved_model.save(model_P2_reshaped, "tf/model_P2_reshaped")
i1 = tf.random.uniform([32, SEQ_LEN, 80], dtype=tf.float32)
i2 = tf.random.uniform([32, TOT_SEQ_LEN, 80], dtype=tf.float32)
# saveTensor([i1, i2], "P2_reshaped", "tf")
model_P2_reshaped(i1, i2)

# model_P3_not_first_reshaped = PhiDecodeP3_not_first_reshaped_quant_alt()
# tf.saved_model.save(model_P3_not_first_reshaped, "tf/model_P3_not_first_reshaped")
# i1 = tf.random.uniform([32, 1, SEQ_LEN], dtype=tf.float32)
# i2 = tf.random.uniform([32, SEQ_LEN, 80], dtype=tf.float32)
# saveTensor([i1, i2], "P2_not_first_reshaped", "tf")
# model_P3_not_first_reshaped(i1, i2)

model_P3_reshaped = PhiDecodeP3_reshaped_quant()
tf.saved_model.save(model_P3_reshaped, "tf/model_P3_reshaped")
i1 = tf.random.uniform([32, SEQ_LEN, TOT_SEQ_LEN], dtype=tf.float32)
i2 = tf.random.uniform([32, TOT_SEQ_LEN, 80], dtype=tf.float32)
# saveTensor([i1, i2], "P3_reshaped", "tf")
model_P3_reshaped(i1, i2)

# model_P3_first_buffered = PhiDecodeP3_first_buffered_quant()
# tf.saved_model.save(model_P3_first_buffered, "tf/model_P3_first_buffered")
# i1 = tf.random.uniform([32, MAX_SEQ_LEN, MAX_SEQ_LEN], dtype=tf.float32)
# i2 = tf.random.uniform([32, MAX_SEQ_LEN, 80], dtype=tf.float32)
# saveTensor([i1, i2], "P3_first_buffered", "tf")
# i1 = loadTensor("/home/kernal1/QM_Sandbox/htp/order66/value_states.bin", [32, MAX_SEQ_LEN, 80], np.float32)
# i2 = loadTensor("/home/kernal1/QM_Sandbox/htp/order66/attn_weights.bin", [32, MAX_SEQ_LEN, MAX_SEQ_LEN], np.float32)
# print(i1)
# print(i2)
# print(model_P3_first_buffered(i1, i2))

# model_P3_not_first_reshaped = PhiDecodeP3_not_first_reshaped_quant()
# # tf.saved_model.save(model_P3_not_first_reshaped, "tf/model_P3_not_first_reshaped")
# i1 = tf.random.uniform([TOT_SEQ_LEN, 32, one], dtype=tf.float32)
# i2 = tf.random.uniform([TOT_SEQ_LEN, 32, 80], dtype=tf.float32)
# # saveTensor([i1, i2], "P3_not_first_reshaped", "tf")
# model_P3_not_first_reshaped(i1, i2)['attn_output'].shape

# model_P3_not_first_buffered = PhiDecodeP3_not_first_buffered_quant()
# # tf.saved_model.save(model_P3_not_first_buffered, "tf/model_P3_not_first_buffered")
# i1 = tf.random.uniform([32, one, MAX_SEQ_LEN], dtype=tf.float32)
# i2 = tf.random.uniform([32, MAX_SEQ_LEN, 80], dtype=tf.float32)
# # saveTensor([i1, i2], "P3_not_first_buffered", "tf")
# model_P3_not_first_buffered(i1, i2)['attn_output'].shape

# for i in range(DECODER_LAYERS):
# for i in range(*DECODER_RANGE):
    # model_P4_1_reshaped = PhiDecodeP4_1_reshaped_quant(params['decoder_layers'][i], config)
    # tf.saved_model.save(model_P4_1_reshaped, "tf/model_P4_1_reshaped_layer_" + str(i))
    # i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
    # saveTensor([i1], "P4_1_reshaped", "tf")
    # model_P4_1_reshaped(i1)

# model_P4_2_reshaped = PhiDecodeP4_2_reshaped_unquant(config)
# # tf.saved_model.save(model_P4_2_reshaped, "tf/model_P4_2_reshaped")
# i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
# i2 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
# i3 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
# # saveTensor([i1, i2, i3], "P4_2_reshaped", "tf")
# model_P4_2_reshaped(i1, i2, i3)

# model_Final_LM_Head = PhiFinalLMHead_quant(params, config)
# tf.saved_model.save(model_Final_LM_Head, "tf/model_Final_LM_Head")
# i1 = tf.random.uniform([SEQ_LEN, HIDDEN_SIZE], dtype=tf.float32)
# model_Final_LM_Head(i1)


INFO:tensorflow:Assets written to: tf/model_P2_reshaped/assets
INFO:tensorflow:Assets written to: tf/model_P3_reshaped/assets


INFO:tensorflow:Assets written to: tf/model_P3_reshaped/assets


{'attn_output': <tf.Tensor: shape=(11, 32, 80), dtype=float32, numpy=
 array([[[3.5667095, 3.3553202, 3.962256 , ..., 3.6162434, 2.5772705,
          2.766594 ],
         [3.4241862, 2.9515045, 2.1299298, ..., 2.5272508, 3.7966962,
          2.7136598],
         [4.175536 , 4.624282 , 3.533833 , ..., 3.206243 , 4.4349113,
          2.8653176],
         ...,
         [3.659787 , 2.750053 , 3.6893284, ..., 3.777901 , 3.3547568,
          3.659802 ],
         [1.9702328, 2.30117  , 1.8625683, ..., 2.6677506, 1.7995511,
          2.07722  ],
         [2.6265805, 2.9811764, 2.14341  , ..., 3.0121865, 3.561525 ,
          3.7235084]],
 
        [[4.2441087, 3.5411332, 3.4328122, ..., 3.3330834, 2.1473715,
          4.0055513],
         [3.403041 , 2.6335218, 2.474135 , ..., 2.1179957, 3.4421794,
          3.0043902],
         [3.187975 , 4.1446266, 2.883797 , ..., 2.6484537, 3.7617135,
          1.8635944],
         ...,
         [2.0297165, 2.062406 , 2.0461736, ..., 2.2687285, 2.7863765,
 